In [1]:
import sys
sys.path.append("../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F

import glob
import os
from datetime import datetime
import time
import math
from tqdm import tqdm

from itertools import repeat
from torch.nn.parameter import Parameter
import collections
import matplotlib
from torch_utils import *
from ExplicitModels import *
from visualization import *
# matplotlib.use('Agg')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                            torchvision.transforms.Normalize(mean=(0.0,), std=(1.0,))])

mnist_dset_train = torchvision.datasets.MNIST('./data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(mnist_dset_train, batch_size=20, shuffle=True, num_workers=0)

mnist_dset_test = torchvision.datasets.MNIST('./data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(mnist_dset_test, batch_size=20, shuffle=False, num_workers=0)

In [4]:
# activation = torch.sigmoid
def mysoftmax(x, dim = 0):
    return F.softmax(x, dim)

def kWTA(inp_vector, k = 1):
    tmpx = torch.clone(inp_vector)
    topval = tmpx.topk(k, dim=0)[0][k - 1]
    topval = topval.repeat(tmpx.shape[0], 1)
    comp = (inp_vector>=topval)
    return comp*inp_vector

def Sparsify_kWTA(inp_vector, sparsity = 0.1):
    k = int(inp_vector.shape[0] * sparsity)
    if k == 0:
        k = 1
    tmpx = torch.clone(inp_vector)
    topval = tmpx.topk(k, dim=0)[0][k - 1]
    topval = topval.repeat(tmpx.shape[0], 1)
    comp = (inp_vector>=topval)
    return comp*inp_vector

activation = Sparsify_kWTA

architecture = [784, 500, 10]
lambda_ = 0.9999
epsilon = 0.01
neural_lr_start = 0.1
neural_lr_stop = 0.00
neural_lr_rule = "constant"
neural_lr_decay_multiplier = 0.005
neural_dynamic_iterations = 50

model = CorInfoMaxBiDirectional_wAutoGrad(architecture, lambda_, epsilon, activation, optimizer_type = "sgd",
                                          optim_lr_ff = 0.01, optim_lr_fb = 0.001, 
                                          use_stepLR = True, sgd_weight_decay = 0.001, 
                                          stepLR_step_size = 10*3000)

In [5]:
trn_acc_list = []
tst_acc_list = []

n_epochs = 50

for epoch_ in range(n_epochs):

    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        #x = activation_inverse(x.view(x.size(0),-1).T, "sigmoid")
        x = x.view(x.size(0),-1).T
        y_one_hot = 2*F.one_hot(y, 10).to(device).T
        #y_one_hot = 0.94 * y_one_hot + 0.03 * torch.ones(*y_one_hot.shape, device = device)
        
        model.batch_step(  x, y_one_hot, neural_lr_start, neural_lr_stop, neural_lr_rule,
                           neural_lr_decay_multiplier, neural_dynamic_iterations,
                         )

    trn_acc = evaluatePC(model, train_loader, device, False, 
                         printing = False)
    tst_acc = evaluatePC(model, test_loader, device, False, 
                         printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))

3000it [04:16, 11.69it/s]
1it [00:00,  9.32it/s]

Epoch : 1, Train Accuracy : 0.8989166666666667, Test Accuracy : 0.9052


3000it [04:16, 11.68it/s]
1it [00:00,  9.45it/s]

Epoch : 2, Train Accuracy : 0.90565, Test Accuracy : 0.9077


3000it [04:20, 11.53it/s]
1it [00:00,  8.86it/s]

...

Epoch : 5, Train Accuracy : 0.9177166666666666, Test Accuracy : 0.9198


3000it [04:22, 11.44it/s]
1it [00:00,  9.06it/s]

Epoch : 6, Train Accuracy : 0.9178666666666667, Test Accuracy : 0.916


76it [00:07,  9.86it/s]


KeyboardInterrupt: 

In [6]:
def accuracy_topk(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].flatten().float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
        
    result = [res_.item() for res_ in res]
    if len(result) == 1:
        result = result[0]
    return result

def evaluatePC_topk(model, loader, device, apply_activation_inverse = True, 
                    activation_type = "sigmoid", topk = [2], printing = True):
    # Evaluate Predictive Coding Model on Classification Task
    correct=0
    phase = 'Train' if loader.dataset.train else 'Test'
    
    for x, y in loader:
        if apply_activation_inverse:
            x = activation_inverse(x.view(x.size(0),-1).T, activation_type).to(device)
        else:
            x = x.view(x.size(0),-1).T.to(device)
        batch_size = x.size(1)
        y = y.to(device)
        
        neurons = model.fast_forward(x)

        #pred = torch.argmax(neurons[-1], dim=0).squeeze()  
        pred = neurons[-1]
        correct += accuracy_topk(pred.T, y, topk) * batch_size

    acc = correct/len(loader.dataset) 
    if printing:
        print(phase+' accuracy :\t', acc)   
    return acc

In [7]:
trn_acc = evaluatePC_topk(model, train_loader, device, False, 
                     printing = True, topk = [2])
tst_acc = evaluatePC_topk(model, test_loader, device, False, 
                     printing = True, topk = [2])

Train accuracy :	 96.355
Test accuracy :	 96.46


In [ ]:
x, y = next(iter(train_loader))
x, y = x.to(device), y.to(device)
x = x.view(x.size(0),-1).T
target = y
output = model.fast_forward(x)[-1].T
mytopk = (2,)
maxk = max(mytopk)
batch_size = target.size(0)
_, pred = output.topk(maxk, 1, True, True)
pred = pred.t()
correct = pred.eq(target.view(1, -1).expand_as(pred))
pred, y

In [ ]:
output[8,:]

In [ ]:
correct[:2].flatten().float().sum(0)

In [ ]:
target.view(1, -1).expand_as(pred), pred

In [ ]:
x, y = next(iter(train_loader))
x, y = x.to(device), y.to(device)
x = x.view(x.size(0),-1).T
pred = model.fast_forward(x)[-1]
accuracy_topk(pred.T, y, [1])

In [ ]:
torch.argmax(model.fast_forward(x)[-1], 0), y

In [ ]:
model.fast_forward(x)[-1][:,6]

In [ ]:
plt.imshow(torch2numpy(model.B[0]['weight']))

In [ ]:
torch.norm(model.Wfb[1]['weight'] - torch.eye(*model.Wfb[1]['weight'].shape).to(device))

In [ ]:
model.Wfb[1]['weight']

In [ ]:
plt.plot(model.neural_dynamics_loss_list)

In [ ]:
model.fast_forward(x)

In [ ]:
sparsity = 0.1
int(neurons[0].shape[0] * sparsity)

In [ ]:
topval.shape

In [ ]:
k = 1
tmpx = torch.clone(neurons[0]) # x.view(x.shape[0], -1)
topval = tmpx.topk(k, dim=0)[0][0]#tmpx.topk(k, dim=0)[0][:,-1]
topval = topval.repeat(tmpx.shape[0], 1)#.permute(1,0)
# comp = (neurons[0]>=topval)


In [ ]:
def kWTA(inp_vector, k = 3):
    tmpx = torch.clone(inp_vector)
    topval = tmpx.topk(k, dim=0)[0][k - 1]
    topval = topval.repeat(tmpx.shape[0], 1)
    comp = (inp_vector>=topval)
    return comp*inp_vector, topval

In [ ]:
neurons[0]

In [ ]:
aa, topval = kWTA(neurons[0], 5)
torch.sum(aa[:,0] != 0)

In [ ]:
neurons[0].topk(3, 0)

In [ ]:
topval.shape

In [ ]:
tmpx.topk(k, dim=0)[0][0]

In [ ]:
torch.max(neurons[0][:,0])

In [ ]:
topval

In [ ]:
topval.repeat(tmpx.shape[1], 1).permute(1,0).shape#.view_as(neurons[0])

In [ ]:
def kWTA(inp_vector, k = 1):
    tmpx = torch.clone(inp_vector) # x.view(x.shape[0], -1)
    topval = tmpx.topk(k, dim=1)[0][:,-1]
#     topval = topval.repeat(tmpx.shape[1], 1).permute(1,0)#.view_as(x)
    topval = topval.repeat(tmpx.shape[1], 1).T
    comp = (inp_vector>=topval)#.to(x)
    return comp*inp_vector

In [ ]:
neurons[0]

In [ ]:
kWTA(neurons[0])[:,0]

In [ ]:
neurons[0]

In [ ]:
kWTA(neurons[0])

In [ ]:
x, y = next(iter(train_loader))
x = x.view(x.size(0),-1).T
x, y = x.to(device), y.to(device)
y = F.one_hot(y, 10).to(device).T
neurons = model.init_neurons(x.size(1))

mode = "train"
if mode == "train":
    neurons[-1] = y.to(torch.float)


neurons = model.run_neural_dynamics( x, y, neurons, neural_lr_start, neural_lr_stop, lr_rule = neural_lr_rule,
                                    lr_decay_multiplier = neural_lr_decay_multiplier, 
                                    neural_dynamic_iterations = neural_dynamic_iterations)

In [ ]:
neurons

In [ ]:
# F.softmax(neurons[0], 0)[:,0]

In [ ]:
# torch.sum(F.softmax(neurons[0], 0), 0)

In [ ]:
# optim_params = []
# for idx in range(len(model.Wff)):
#     for key_ in ["weight", "bias"]:
#         optim_params.append(  {'params': model.Wff[idx][key_], 'lr': lr_start["ff"]}  )

In [ ]:
# optimizer = torch.optim.Adam(optim_params, maximize = True)

In [ ]:
trn_acc_list = []
tst_acc_list = []

n_epochs = 50

for epoch_ in range(n_epochs):
    if epoch_ > 12:
        neural_lr_start = 0.05
    if epoch_ > 17:
        neural_lr_start = 0.03
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        #x = activation_inverse(x.view(x.size(0),-1).T, "sigmoid")
        x = x.view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T
        y_one_hot = 0.94 * y_one_hot + 0.03 * torch.ones(*y_one_hot.shape, device = device)
        
        model.batch_step(  x, y_one_hot, neural_lr_start, neural_lr_stop, neural_lr_rule,
                           neural_lr_decay_multiplier, neural_dynamic_iterations,
                         )

    trn_acc = evaluatePC(model, train_loader, device, False, 
                         printing = False)
    tst_acc = evaluatePC(model, test_loader, device, False, 
                         printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))

In [ ]:
model.Wff